In [ ]:
import sys
sys.path.append('../src/')

import warnings
#warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

import os
from pathlib import Path
import json
import pickle

from session import Session
from helpers import *

In [ ]:
FAMILY_ID = 4
SEANCE_ID = 1
SESSION_ID = 0
session_pathname = Path(f"../data/output/processed/session_{SESSION_ID}_family_{FAMILY_ID}_seance_{SEANCE_ID}.pkl")

with open(session_pathname, "rb") as f:
    session = pickle.load(f)
print(f"Session loaded from {session_pathname}")

for pr in session.physio_recordings:
    print(f"Signal epoched for subject {pr.subject_id}: {pr.data_epoched}")